In [ ]:
import pandas as pd
from api_conection.query import main as consultar_dni


In [ ]:
# Paso 1: Cargar la base de datos original y crear una copia
df_original = pd.read_excel("data/db_original/Base_Etna_2.12.24-1.xlsx", sheet_name="Base_Etna_2.12")
df_copia = df_original.copy() # Crear una copia de la base

In [ ]:
# Crear un DataFrame para los resultados finales
resultados = []

# Función para comparar al menos 4 caracteres consecutivos
def comparar_4_caracteres_consecutivos(texto_original, texto_corregido):
    texto_original = str(texto_original)
    texto_corregido = str(texto_corregido)
    for i in range(len(texto_original) - 3):
        if texto_original[i:i+4] == texto_corregido[i:i+4]:
            return "Coincide"
    return "No coincide"

# Paso 2: Iterar sobre los DNIs y procesar la información
for index, row in df_original.iterrows():
    dni_original = str(row["DNI"]) # DNI original
    digitos_dni = len(dni_original)  # Contar los dígitos reales del DNI
    dni_consulta = dni_original.zfill(8)  # Agregar ceros iniciales para consultar la API
    nombre_db = row["Nombre"]
    apellido_db = row["Apellido"]
    observaciones = "-"

    # Validar DNIs con más de 8 dígitos
    if digitos_dni > 8:
        observaciones = "Más de 8 dígitos"
        resultados.append({
            "DNI": dni_original,
            "nombre_db": nombre_db,
            "nombre_corroborado_api": "-",
            "apellido_db": apellido_db,
            "apellido_corroborado_api": "-",
            "coincidencia_8_digitos": "-",
            "coincidencia_7_digitos": "-",
            "coincidencia_6_digitos": "-",
            "coincidencia_5_digitos": "-",
            "observaciones": observaciones
        })
        continue

    print(f"Consultando DNI: {dni_consulta}...")

    try:
        # Llamar a la API para obtener datos
        resultado = consultar_dni(dni_consulta)
        if resultado:
            nombre_api = resultado.get("nombres", "")
            apellido_api = f"{resultado.get('apellidoPaterno', '')} {resultado.get('apellidoMaterno', '')}"
        else:
            nombre_api = nombre_db
            apellido_api = apellido_db
            observaciones = "No se encontró información sobre el DNI"
    except Exception as e:
        print(f"Error al consultar el DNI {dni_consulta}: {e}")
        nombre_api = nombre_db
        apellido_api = apellido_db
        observaciones = "Error al consultar la API"

    # Comparar nombres y apellidos
    coincidencia_nombre = comparar_4_caracteres_consecutivos(nombre_db.upper(), nombre_api.upper())
    coincidencia_apellido = comparar_4_caracteres_consecutivos(apellido_db.upper(), apellido_api.upper())
    coincidencia_total = f"{coincidencia_nombre}, {coincidencia_apellido}"

    # Crear las columnas de coincidencias dinámicamente
    coincidencias = {
        "coincidencia_8_digitos": "-",
        "coincidencia_7_digitos": "-",
        "coincidencia_6_digitos": "-",
        "coincidencia_5_digitos": "-"
    }
    coincidencias[f"coincidencia_{digitos_dni}_digitos"] = coincidencia_total  # Agregar coincidencia en la columna correspondiente

    # Agregar los resultados al DataFrame final
    resultados.append({
        "DNI": dni_original,
        "nombre_db": nombre_db,
        "nombre_corroborado_api": nombre_api,
        "apellido_db": apellido_db,
        "apellido_corroborado_api": apellido_api,
        **coincidencias,
        "observaciones": observaciones
    })

# Paso 3: Crear el DataFrame final con los resultados
df_resultados = pd.DataFrame(resultados)

# Ordenar las columnas
columnas_ordenadas = [
    "DNI", "nombre_db", "nombre_corroborado_api", "apellido_db", "apellido_corroborado_api",
    "coincidencia_8_digitos", "coincidencia_7_digitos", "coincidencia_6_digitos", "coincidencia_5_digitos",
    "observaciones"
]
df_resultados = df_resultados[columnas_ordenadas]

# Paso 4: Exportar el DataFrame final a un archivo Excel
archivo_final = "elrayo/data/db_pruebas/db_corroborado.xlsx"
df_resultados.to_excel(archivo_final, index=False)
print(f"Archivo final exportado a {archivo_final}")